In [ ]:
import CNN_lib
import CNN_utilities
import fig_lib 
import CNN_machine

import numpy as np
import pickle
import json

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# SETTINGS

In [ ]:
trains_n=5

data_split='rnd_patients'                                # 'rnd_recs' 'rnd_patients'

checking_on=False
t_btw_frm=0.05
check_record_1= '670615_19'
check_record_2= 'ROT_670615_19'

In [ ]:
#number_data_ls = [i for i in range(48, 530, 32)]
number_data_ls = [i for i in range(48, 530, 170)]
models_d = {'model'+'_'+str(n):{'path':None,'name':None,'samples':n} for n in number_data_ls}

epochs = 4

models_d

### DATA SPLIT

In [ ]:
if data_split == 'rnd_recs': test_size=0.2           

elif data_split == 'rnd_patients': patiens_split =[12,6]

### DATA AUGMENTATION

In [ ]:
n_flip_1,n_flip_0=0.7 , 0.07

n_rot_1,n_rot_0=0.7 , 0.07

n_contr_1,n_contr_0=0.4 , 0.04

n_bright_1,n_bright_0=0.4 , 0.04

aug_params={'f1':n_flip_1,'f0':n_flip_0,'r1':n_rot_1,'r0':n_rot_0,'c1':n_contr_1,'c0':n_contr_0,'b1':n_bright_1,'b0':n_bright_0}

### PATHS

In [ ]:
checkpoint_path = "C:/PROJECTS\emboendo\CNN\python-machine-learning\emboendo_ML\embo_CNN_lab\_static/AUC_VS_SAMPLES_CHECKPOINTS/chP"
pkl_train_p='C:\PROJECTS\emboendo\CNN/pikles/processed_train_d.pkl'  

for n in number_data_ls:
    
    models_d['model'+'_'+str(n)]['path']=checkpoint_path+'_'+str(n)
    models_d['model'+'_'+str(n)]['name']='MODEL'+ ' ' + str(n)

check_points_p = [checkpoint_path+'_'+str(n)+'.h5' for n in range(trains_n)]

if data_split == 'rnd_patients': json_fix_sq_p='C:\PROJECTS\emboendo\dicom_viewer\_static\Jsons\points_fix_d.json'

# GENERATE TRAIN DATA

In [ ]:
if data_split == 'rnd_patients':

    with open(json_fix_sq_p, 'r') as json_file:
        json_d = json.load(json_file)

    patients_d_df =  CNN_utilities.gen_patients_d_df(json_d)

In [ ]:
with open(pkl_train_p, 'rb') as pikle_file:

    save_trains= pickle.load(pikle_file)

x_train,y_train,rcs =  save_trains[0],save_trains[1],save_trains[2]

max_frm_n, HEIGHT, WIDTH=np.shape(x_train)[1],np.shape(x_train)[2],np.shape(x_train)[3]
input_shape=(max_frm_n, HEIGHT, WIDTH, 1)

## MAIN DATA SHUFFLE

In [ ]:
X_train,Y_train,recs=CNN_lib.shuffle(x_train,y_train,rcs)

# CNN

In [ ]:
models=[CNN_machine.Gen_Model(aug_params,patients_d_df,name=values['name'],path=values['path'],input_shape=input_shape,samples=values['samples']) for values in models_d.values()]

In [ ]:
for mdl in models:
    
    print(f'---------------------------------- ---------------------------------- ---------- ---------------------------------- ----------------------------------')
    print(f'---------------------------------- ---------------------------------- {mdl.name} ---------------------------------- ----------------------------------')
    print(f'---------------------------------- ---------------------------------- ---------- ---------------------------------- ----------------------------------')
    print(' ')

    mdl.train_model(X_train,Y_train,recs, epochs,trains_n,patiens_split)

    print(' ')
    print(' ')

In [ ]:
#for mdl in models:CNN_lib.plot_train_eval(mdl.history,epochs)

In [ ]:
#for mdl in models: CNN_lib.plot_roc_curve(mdl.fpr_val,mdl.tpr_val,mdl.roc_auc)

In [ ]:
for mdl in models:CNN_lib.list_plot(mdl.roc_aucs,title=mdl.name,ylabel='AUCS')

In [ ]:
AUC_ls=[np.mean(mdl.roc_aucs) for mdl in models]
number_points=[mdl.samples for mdl in models]



CNN_lib.simple_plot(number_points,AUC_ls,title='MEAN AUC VS SAMPLES',xlabel='SAMPLES',ylabel='MEAN AUC')